In [1]:
import langchain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.agents import load_tools, initialize_agent
import openai
import os
import wandb
from wandb.integration.langchain import WandbTracer
import json

In [8]:
wandb.init(
    project="generate_simple_prd",
    config={
        "model": "gpt-3.5-turbo",
        "temperature": 0
    },
    entity="arihantsheth",
    name="generate_simple_prd_v3",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arihantsheth. Use `wandb login --relogin` to force relogin


In [2]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

prompt_template = load_prompt("../prompt_templates/generate_prd_template_v3.json")

c:\Users\ariha\Desktop\Synap Labs\synap_labs_env\lib\site-packages\langchain\llms\openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\ariha\Desktop\Synap Labs\synap_labs_env\lib\site-packages\langchain\llms\openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
prompt_input_variables = {
    "wandb": dict(
        new_feature="Model Explainability Toolkit (MXT)",
        new_feature_desc="Enables users to interpret and explain the outputs and decision-making of their AI Models."
    ),
    "langchain": dict(
        new_feature="Model Training Wizard (MTW)",
        new_feature_desc=" A user-friendly interface that guides users through the process of training their own custom language models. It would provide step-by-step instructions, dataset management tools, and performance evaluation metrics to help users create high-quality models tailored to their specific needs."
    ),
    "dualcam": dict(
        new_feature="Dual Camera Activation Button",
        new_feature_desc="Enables users to capture photos from both front and back cameras simultaneously."
    )
}

In [4]:
string_template = """\
You are a talkative product manager.

Now, you want to add the following new feature:
{new_feature}
Feature description: {new_feature_desc}

Create a Product Requirement Document (PRD) to add this new feature in the following format. Each heading contains a description of what should be included in that section. Return your output in a markdown format to maintain consistency:

1. What Are We Solving For?
    Give the high-level reasoning for this proposal. What is the problem or opportunity we’re pursuing? Who is this for? What segments or personas are impacted?

2. Why Does It Matter?
    Why is this initiative big enough to matter to Customers? What data or studies do we have to support this? 

3. What Does Success Look Like?
    What is the primary metric you’ll use to gauge success of this initiative? What is the target lift/impact to this metric, over what time period?
    What are the secondary metric(s) of success?

4. Background
    Include any necessary background research here, including data analysis, industry research, competitor analysis, quantitative or qualitative customer insights, etc.

5. Known Constraints
    Where applicable, lay out any constraints that must be taken into account when designing a solution. These constraints may related to usability (i.e. accessibility), time (seasonality), technology (platform capabilities or site performance), legal, and business guardrails. 

6. Solution Proposal
    Given the opportunity, the background, and the constraints, what are we proposing? List bullet points or insert sketches that describe the solution at a high level. If this initiative is successful, what will get better for the customers?

7. Risks & Assumptions
    What must we assume to be true in order for this initiative to be successful? What developments could cause this initiative to be unsuccessful? Risks include fraud/abuse potential, security…

8. Target Customers
>   8.1 Market
        Detail the specific regions & countries which the Product Development Team needs to build a solution for? Where we can lean into opportunities? Where do we need to mitigate emergent risks?
>   8.2 Audience Segments
        List the unique customer audiences this solution needs to address uniquely. 

9. User Experience
>   9.1 User Flows
        What screens, touch points or experiences that are being considered as part of this initiative? There may be several - and these may cross boundaries of products, experiences and user states. 

10. High Level Use Cases: Fill it in the given format

|    	| Name                                                       	| Description                                                                                                                                            	| PDT or Squad                                                                           	| Dependency?                                                                 	| Priority                                    	| Open Questions,  Notes & Discussions                                             	|
|----	|------------------------------------------------------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------	|----------------------------------------------------------------------------------------	|-----------------------------------------------------------------------------	|---------------------------------------------	|----------------------------------------------------------------------------------	|
| Desc  | Give this use case a name. e.g. Order Lifecycle Management 	| Note: this is not a user story. User stories are much more detailed/granular views of use cases. They represent the requirements & definition of done. 	| Name the Product Development team or squad if you know who will work on this use case. 	| If this use case has a dependency, list the use case # on which it depends. 	| MUST HAVE SHOULD HAVE COULD HAVE WON'T HAVE 	| Important things team members should know / things you're working to figure out. 	|

11. Rollout Approach
    How will we execute this rollout? Big-bang or gradual scale-up? A/B test or launch-and-measure? 
    Will the rollout happen in every market at once, or will markets ramp up one by one?
    Be inclusive of other key Product Development Team members in having this discussion, in particular Data Analysts, Data Science and Business Ops, along with Developers.
    Provide a comprehensive view into the Pros/Cons, what can/cannot be answered as a result, and reasoning behind the decision being made for the rollout approach. 

12. Measuring Success
    If A/B testing, describe the variants being tested, traffic allocations to each variant, and the KPIs being measured in the experiment. 
    Describe the desired cadence of performance readouts and align with Analytics to ensure it’s covered.
    As with the Rollout Approach section, please be sure to socialize this plan with your partners in Analytics, Data Science and Business Operations.

13. Competitive Analysis:
    List 10 competitors of this application. Create a detailed competitive analysis report comparing the above applications. include the following points: user base, user region, different features supported, and pricing tiers. Don't limit yourself to these categories and think of other categories yourself. Return the output in a well-structured Markdown table. Think step by step"""

In [28]:
prompt_template = PromptTemplate(template=string_template, input_variables=["new_feature", "new_feature_desc"])

In [18]:
# prompt_template.save("../prompt_templates/generate_prd_template_v3.json")

In [5]:
prompts = dict()
for company in prompt_input_variables:
    prompts[company] = prompt_template.format(**prompt_input_variables[company])

In [6]:
with open("../company_prompts/prompt_inputs_v3.json", "w") as f:
    json.dump(prompt_input_variables, f)

with open("../company_prompts/prompts_v3.json", "w") as f:
    json.dump(prompts, f)

In [30]:
outputs = dict()

for company in prompts:
    print(f"Generating PRD for {company}...")
    prompt = prompts[company]
    
    try:
        output = llm(prompt, callbacks=[WandbTracer()])
        # output = llm(prompt)
    except Exception as e:
        print(e)
        output = f"ERROR: {e}"

    outputs[company] = output
    wandb.log({f"{company}_prd": output})

Generating PRD for dualcam...
The model: `gpt-4` does not exist


In [77]:
for company in outputs:
    with open(f"../generated_prds/{company}_prd_v3.md", "w") as f:
        f.write(outputs[company])

In [66]:
wandb.finish()

langchain_prd,# Product Requiremen...
wandb_prd,# Product Requiremen...


In [22]:
print(prompts["dualcam"])

You are a talkative product manager.

Now, you want to add the following new feature:
Dual Camera Activation Button
Feature description: Enables users to capture photos from both front and back cameras simultaneously.

Create a Product Requirement Document (PRD) to add this new feature in the following format. Each heading contains a description of what should be included in that section. Return your output in a markdown format to maintain consistency:

1. What Are We Solving For?
    Give the high-level reasoning for this proposal. What is the problem or opportunity we’re pursuing? Who is this for? What segments or personas are impacted?

2. Why Does It Matter?
    Why is this initiative big enough to matter to Customers? What data or studies do we have to support this? 

3. What Does Success Look Like?
    What is the primary metric you’ll use to gauge success of this initiative? What is the target lift/impact to this metric, over what time period?
    What are the secondary metric(s

In [21]:
from IPython.display import Markdown, display

display(Markdown(outputs["dualcam"]))

# Product Requirement Document (PRD) - Dual Camera Activation Button

## 1. What Are We Solving For?
We are proposing the addition of a Dual Camera Activation Button feature that enables users to capture photos from both front and back cameras simultaneously. This feature aims to provide users with the ability to capture moments from different perspectives in a single shot. It is targeted towards smartphone users who enjoy photography and want to enhance their photo-taking experience.

## 2. Why Does It Matter?
This initiative matters to customers as it offers a unique and convenient way to capture photos using both front and back cameras simultaneously. It allows users to create more dynamic and creative shots, enhancing their photography experience. Although there is no specific data or studies available to support this feature, it aligns with the growing trend of smartphone photography and the demand for innovative camera features.

## 3. What Does Success Look Like?
The primary metric to gauge the success of this initiative would be the increase in user engagement with the dual camera feature. The target lift for this metric would be a 20% increase in the number of photos captured using the dual camera activation button within the first three months of its implementation. 

The secondary metrics of success would include an increase in user satisfaction with the camera functionality, positive user feedback, and an increase in user retention rate.

## 4. Background
No specific background research or data analysis is available for this feature proposal. However, the popularity of dual camera setups in smartphones and the demand for advanced camera features indicate a potential market interest in this functionality.

## 5. Known Constraints
- Usability: The solution should ensure accessibility and ease of use for all users, including those with disabilities.
- Technology: The availability and capabilities of dual camera setups may vary across different smartphone models and operating systems. The solution should be designed to work seamlessly with a wide range of devices.
- Legal: The solution should comply with privacy regulations and ensure that users' privacy is protected during the photo capture process.

## 6. Solution Proposal
- Add a dedicated Dual Camera Activation Button within the camera app interface.
- Enable users to switch between single camera mode and dual camera mode with a single tap.
- Provide an intuitive user interface that allows users to preview and adjust the composition of both front and back camera views simultaneously.
- Implement automatic synchronization of exposure and focus settings between the front and back cameras.
- Allow users to capture photos using both cameras simultaneously by tapping the Dual Camera Activation Button.

If this initiative is successful, customers will have a more versatile and creative tool for capturing photos, resulting in more engaging and visually appealing images.

## 7. Risks & Assumptions
- Assumption: Users will find value in capturing photos from both front and back cameras simultaneously.
- Risk: The dual camera activation feature may not be compatible with all smartphone models or operating systems, limiting its adoption.
- Risk: Users may find the dual camera activation process confusing or cumbersome, leading to a decrease in user satisfaction.

## 8. Target Customers
### 8.1 Market
The Product Development Team needs to build a solution that caters to smartphone users globally. The solution should be designed to work across different regions and countries, leveraging opportunities in the growing smartphone photography market.

### 8.2 Audience Segments
- Photography enthusiasts who want to explore new creative possibilities.
- Social media users who want to capture more engaging and visually appealing photos for sharing.
- Travelers who want to capture both their surroundings and their own reactions simultaneously.

## 9. User Experience
### 9.1 User Flows
- User opens the camera app.
- User taps the Dual Camera Activation Button to switch to dual camera mode.
- User adjusts the composition and settings for both front and back cameras.
- User taps the capture button to take a photo using both cameras simultaneously.

## 10. High Level Use Cases:

|    	| Name                                                       	| Description                                                                                                                                            	| PDT or Squad                                                                           	| Dependency?                                                                 	| Priority                                    	| Open Questions,  Notes & Discussions                                             	|
|----	|------------------------------------------------------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------	|----------------------------------------------------------------------------------------	|-----------------------------------------------------------------------------	|---------------------------------------------	|----------------------------------------------------------------------------------	|
| 1 	| Dual Camera Activation                                     	| Allows users to capture photos from both front and back cameras simultaneously.                                                                         	| Camera Development Team                                                                	| -                                                                           	| MUST HAVE                                    	| -                                                                                	|
| 2 	| Dual Camera Composition Adjustment                         	| Enables users to adjust the composition and settings for both front and back cameras in dual camera mode.                                              	| Camera Development Team                                                                	| 1 (Dual Camera Activation)                                                   	| MUST HAVE                                    	| -                                                                                	|
| 3 	| Dual Camera Synchronization                                	| Implements automatic synchronization of exposure and focus settings between the front and back cameras in dual camera mode.                           	| Camera Development Team                                                                	| 1 (Dual Camera Activation)                                                   	| MUST HAVE                                    	| -                                                                                	|
| 4 	| Dual Camera Capture                                         	| Allows users to capture photos using both front and back cameras simultaneously by tapping the Dual Camera Activation Button.                          	| Camera Development Team                                                                	| 1 (Dual Camera Activation)                                                   	| MUST HAVE                                    	| -                                                                                	|

## 11. Rollout Approach
The rollout approach for this feature will be a gradual scale-up. The feature will be initially launched as an A/B test in a selected market to evaluate its performance and gather user feedback. Based on the results and feedback, the rollout will be expanded to additional markets.

The A/B test will involve randomly assigning a portion of users to the experimental group, which will have access to the dual camera activation feature, while the control group will not have access. The performance will be measured based on user engagement metrics, user satisfaction surveys, and feedback.

## 12. Measuring Success
In the A/B test, the experimental variant will include the dual camera activation feature, while the control variant will not have access to it. The traffic allocation will be 50% for each variant.

The key performance indicators (KPIs) to be measured in the experiment include:
- Increase in the number of photos captured using the dual camera activation button
- User engagement metrics (e.g., time spent in the camera app, frequency of camera usage)
- User satisfaction surveys and feedback

Performance readouts will be conducted on a weekly basis, and the results will be shared with the Analytics, Data Science, and Business Operations teams.

## 13. Competitive Analysis:

| Application 	| User Base 	| User Region 	| Different Features Supported                                                                                                                            	| Pricing Tiers                                                                                         	|
|-------------	|-----------	|-------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------	|-------------------------------------------------------------------------------------------------------	|
| App 1       	| 10 million 	| Global      	| - Dual camera support<br>- Advanced editing tools<br>- Social media integration<br>- Filters and effects<br>- Photo sharing<br>- In-app purchases      	| Free<br>- Premium: $4.99/month<br>- Pro: $9.99/month                                                 	|
| App 2       	| 5 million  	| North America 	| - Dual camera support<br>- Live photo mode<br>- AR filters<br>- Collage maker<br>- Photo editing<br>- Cloud storage<br>- Social media sharing             	| Free<br>- Basic: $2.99/month<br>- Premium: $6.99/month                                             	|
| App 3       	| 8 million  	| Europe      	| - Dual camera support<br>- Portrait mode<br>- AI-powered filters<br>- Photo editing<br>- Photo albums<br>- Social media integration<br>- In-app purchases 	| Free<br>- Premium: $3.99/month<br>- Pro: $7.99/month                                                 	|
| App 4       	| 12 million 	| Asia        	| - Dual camera support<br>- Beauty mode<br>- Stickers and emojis<br>- Photo editing<br>- Photo sharing<br>- In-app purchases                             	| Free<br>- Premium: $2.99/month<br>- Pro: $5.99/month                                                 	|
| App 5       	| 6 million  	| South America 	| - Dual camera support<br>- Panorama mode<br>- Filters and effects<br>- Photo editing<br>- Social media sharing<br>- In-app purchases                    	| Free<br>- Basic: $1.99/month<br>- Premium: $4.99/month                                             	|
| App 6       	| 3 million  	| Africa      	| - Dual camera support<br>- Selfie mode<br>- Photo editing<br>- Photo albums<br>- Social media integration<br>- In-app purchases                         	| Free<br>- Premium: $2.99/month<br>- Pro: $6.99/month                                                 	|
| App 7       	| 7 million  	| Australia   	| - Dual camera support<br>- Slow motion mode<br>- Filters and effects<br>- Photo editing<br>- Photo sharing<br>- In-app purchases                       	| Free<br>- Basic: $3.99/month<br>- Premium: $7.99/month                                             	|
| App 8       	| 9 million  	| Middle East 	| - Dual camera support<br>- Night mode<br>- AR stickers<br>- Photo editing<br>- Photo albums<br>- Social media sharing<br>- In-app purchases            	| Free<br>- Premium: $4.99/month<br>- Pro: $9.99/month                                                 	|
| App 9       	| 4 million  	| Central America 	| - Dual camera support<br>- HDR mode<br>- Filters and effects<br>- Photo editing<br>- Photo sharing<br>- In-app purchases                               	| Free<br>- Basic: $2.99/month<br>- Premium: $5.99/month                                             	|
| App 10      	| 2 million  	| Caribbean   	| - Dual camera support<br>- Panorama mode<br>- Beauty mode<br>- Photo editing<br>- Social media integration<br>- In-app purchases                         	| Free<br>- Premium: $3.99/month<br>- Pro: $8.99/month                                                 	|